In [1]:
import bs4 as bs
from dalab import *
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_results(query, type_=None, stage=None, tech=None):
    
#     tech is possible only if type not None, same for others

    if type_:
        type_ = type_.replace(' ', '+')
        url = 'https://angel.co/companies' + '?company_types[]=' + type_
        if stage:
            url = url + '&stage=' + stage
            if tech:
                url = url + '&teches[]=' + tech
    else:  
        url = 'https://angel.co/companies' 

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
    driver = webdriver.Chrome(options=None)
    driver.set_window_position(3000,0)
    driver.get(url)
    time.sleep(5)
    
    search_box = driver.find_element_by_class_name("search-box")
    search_box.click()
    
    input_bar = driver.find_element_by_class_name('keyword-input')
    input_bar.send_keys(query)
    input_bar.send_keys(Keys.ENTER)
    time.sleep(3)

    while True:
        try:
            source = driver.page_source
            loadMoreButton = driver.find_element_by_class_name('more').click()
            time.sleep(4)
        except: 
            break
            
    driver.close()

    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        result_list =  soup.find_all('div', {'class': 'results'})[0]
        results = result_list.find_all('div', {'data-_tn': 'companies/row'})
    except:
        print('Could not get results')
        return

    
    return results


def parse_results(results):
    df = pd.DataFrame(columns=['kind', 'link', 'name', 'pitch', 'joined', 'location', 'market', 'size', 'stage', 'raised'],
                  index=[0])
    for result in results[1:]:
        try:
            dic = {}
            try:
                dic['kind'] = result.a['data-type']
            except: dic['kind'] = None

            try:
                dic['link'] = result.a['href']
            except: dic['link'] = None

            try:
                dic['name'] = result.a['title']
            except: dic['name'] = None
                
            try:
                dic['website'] = result.find('div', {'data-column': 'website'}).text.split()[1]
            except: dic['website'] = None

            try:
                dic['pitch'] = result.find('div', {'class': 'pitch'}).text
            except: dic['pitch'] = None

            try:
                dic['joined'] = result.find('div', {'data-column': 'joined'}).text.split('Joined')[1].strip()
            except: dic['joined'] = None

            try:
                dic['location'] = result.find('div', {'data-column': 'location'}).text.split('Location')[1].strip()
            except: dic['location'] = None

            try:
                dic['market'] = result.find('div', {'data-column': 'market'}).text.split('Market')[1].strip()
            except: dic['market'] = None

            try:
                dic['size'] = result.find('div', {'data-column': 'company_size'}).text.split()[1]
            except: dic['size'] = None

            try:
                dic['stage'] = result.find('div', {'data-column': 'stage'}).text.split('Stage')[1].strip()
            except: dic['stage'] = None

            try:
                dic['raised'] = result.find('div', {'data-column': 'raised'}).text.split('Raised')[1].strip()
            except: dic['raised'] = None

            df = df.append(pd.DataFrame(dic, index=[0]))

        except: pass

    df = df.reset_index(drop=True)
    return df



def get_companies(query, type_=None, stage=None, tech=None):
    results = get_results(query, type_, stage, tech)
    if results:
        df = parse_results(results)
        return df

In [7]:
queries = ['artificial intelligence', 'machine learning', 'data science', 'big data', 'neural network', 
           'deep learning', 'natural language processing', 'transfer learning', 'reinforcement learning',
           'nlp', 'recurrent neural', 'convolution neural', 'data mining', 'data extraction', 
           'data engineering', 'web scraping', 'web scrape', 'web crawler', 'web crawling',
           'evolutionary computation', 'genetic algorithm', 'genetic programming', 'object recognition',
           'image understanding', 'speech processing', ' speech recognition', 'machine translation', 
           'time series', 'ai', 'intelligent machine', 'pattern recognition', 'computer vision']


types = ['Startup', 'VC Firm', 'Private Company', 'Incubator', 'SaaS', 'Mobile App', None]
stages = ['Seed', 'Series A', 'Series B', 'Series C', 'Acquired', None]
techs = ['Python', 'Javascript', 'HTML5', 'Java', 'CSS', None]


df = pd.DataFrame()
for type_ in types[2:]:
    for stage in stages:
        for tech in techs:
            for query in queries:
                print(query, type_, stage, tech)

                try:
                    companies = get_companies(query, type_, stage, tech)
                    companies['query'] = query
                    companies['type_'] = type_
                    companies['stage'] = stage
                    companies['tech'] = tech
                    print('Done')
                except Exception as e: 
                    companies = pd.DataFrame()
                    print('Could not get companies')
                    print('\n')
                    print(e)
                    print('\n')
                    pass

                try:
                    if df.empty:
                        df = companies
                    else:
                        df = df.append(companies)
                except:
                    print('Unable to append new data.')
                    pass
                    
            df = df.reset_index(drop=True)
            df.to_csv('data/ai_startups4.csv')
            print(df.location.head())
                    
            
df = df.reset_index(drop=True)
df.to_csv('data/ai_startups4.csv')

artificial intelligence Private Company Seed Python
Done
machine learning Private Company Seed Python
Done
data science Private Company Seed Python
Done
big data Private Company Seed Python
Done
neural network Private Company Seed Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


deep learning Private Company Seed Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


natural language processing Private Company Seed Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


transfer learning Private Company Seed Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


reinforcement learning Private Company Seed Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


nlp Private Company Seed Python
Could not get results
Could no

Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence Private Company Seed HTML5
Done
machine learning Private Company Seed HTML5
Could not get results
Could not get companies


'NoneType' object does not support item assignment


data science Private Company Seed HTML5
Could not get results
Could not get companies


'NoneType' object does not support item assignment


big data Private Company Seed HTML5
Done
neural network Private Company Seed HTML5
Could not get results
Could not get companies


'NoneType' object does not support item assignment


deep learning Private Company Seed HTML5
Could not get results
Could not get companies


'NoneType' object does not support item assignment


natural language processing Private Company Seed HTML5
Could not get results
Could not get companies


'NoneType' object does not support item assignment


transfer learning Private Company Seed HT

Done
intelligent machine Private Company Seed Java
Could not get results
Could not get companies


'NoneType' object does not support item assignment


pattern recognition Private Company Seed Java
Could not get results
Could not get companies


'NoneType' object does not support item assignment


computer vision Private Company Seed Java
Could not get results
Could not get companies


'NoneType' object does not support item assignment


0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence Private Company Seed CSS
Done
machine learning Private Company Seed CSS
Could not get results
Could not get companies


'NoneType' object does not support item assignment


data science Private Company Seed CSS
Could not get results
Could not get companies


'NoneType' object does not support item assignment


big data Private Company Seed CSS
Done
neural network Private Company Seed CSS
Could not get

Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


machine learning Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


data science Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


big data Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


neural network Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not support item assignment


deep learning Private Company Series A Python
Could not get results
Could not get companies


'NoneType' object does not s

Could not get companies


Message: element not interactable
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=73.0.3683.20 (8e2b610813e167eee3619ac4ce6e42e3ec622017),platform=Windows NT 10.0.17134 x86_64)



data extraction Private Company Series A Javascript
Could not get companies


Message: element not interactable
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=73.0.3683.20 (8e2b610813e167eee3619ac4ce6e42e3ec622017),platform=Windows NT 10.0.17134 x86_64)



data engineering Private Company Series A Javascript
Could not get companies


Message: element not interactable
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=73.0.3683.20 (8e2b610813e167eee3619ac4ce6e42e3ec622017),platform=Windows NT 10.0.17134 x86_64)



web scraping Private Company Series A Javascript
Could not get companies


Message: element not interactable
  (Session info: chrome=72.0.3626.109)
  (Driver info: chromedriver=73.0.3683.20 (8e2b610813e167eee3619ac4ce

...

Done
speech processing None Series B Python
Done
 speech recognition None Series B Python
Done
machine translation None Series B Python
Done
time series None Series B Python
Done
ai None Series B Python
Done
intelligent machine None Series B Python
Done
pattern recognition None Series B Python
Done
computer vision None Series B Python
Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence None Series B Javascript
Done
machine learning None Series B Javascript
Done
data science None Series B Javascript
Done
big data None Series B Javascript
Done
neural network None Series B Javascript
Done
deep learning None Series B Javascript
Done
natural language processing None Series B Javascript
Done
transfer learning None Series B Javascript
Done
reinforcement learning None Series B Javascript
Done
nlp None Series B Javascript
Done
recurrent neural None Series B Javascript
Done
convolution neur

Done
recurrent neural None Series C Python
Done
convolution neural None Series C Python
Done
data mining None Series C Python
Done
data extraction None Series C Python
Done
data engineering None Series C Python
Done
web scraping None Series C Python
Done
web scrape None Series C Python
Done
web crawler None Series C Python
Done
web crawling None Series C Python
Done
evolutionary computation None Series C Python
Done
genetic algorithm None Series C Python
Done
genetic programming None Series C Python
Done
object recognition None Series C Python
Done
image understanding None Series C Python
Done
speech processing None Series C Python
Done
 speech recognition None Series C Python
Done
machine translation None Series C Python
Done
time series None Series C Python
Done
ai None Series C Python
Done
intelligent machine None Series C Python
Done
pattern recognition None Series C Python
Done
computer vision None Series C Python
Done
0               NaN
1      Santa Monica
2    Silicon Valley
3 

Done
computer vision None Series C None
Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence None Acquired Python
Done
machine learning None Acquired Python
Done
data science None Acquired Python
Done
big data None Acquired Python
Done
neural network None Acquired Python
Done
deep learning None Acquired Python
Done
natural language processing None Acquired Python
Done
transfer learning None Acquired Python
Done
reinforcement learning None Acquired Python
Done
nlp None Acquired Python
Done
recurrent neural None Acquired Python
Done
convolution neural None Acquired Python
Done
data mining None Acquired Python
Done
data extraction None Acquired Python
Done
data engineering None Acquired Python
Done
web scraping None Acquired Python
Done
web scrape None Acquired Python
Done
web crawler None Acquired Python
Done
web crawling None Acquired Python
Done
evolutionary computation None Acquir

Done
web crawler None Acquired None
Done
web crawling None Acquired None
Done
evolutionary computation None Acquired None
Done
genetic algorithm None Acquired None
Done
genetic programming None Acquired None
Done
object recognition None Acquired None
Done
image understanding None Acquired None
Done
speech processing None Acquired None
Done
 speech recognition None Acquired None
Done
machine translation None Acquired None
Done
time series None Acquired None
Done
ai None Acquired None
Done
intelligent machine None Acquired None
Done
pattern recognition None Acquired None
Done
computer vision None Acquired None
Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
artificial intelligence None None Python
Done
machine learning None None Python
Done
data science None None Python
Done
big data None None Python
Done
neural network None None Python
Done
deep learning None None Python
Done
natural language processi

Done
genetic algorithm None None None
Done
genetic programming None None None
Done
object recognition None None None
Done
image understanding None None None
Done
speech processing None None None
Done
 speech recognition None None None
Done
machine translation None None None
Done
time series None None None
Done
ai None None None
Done
intelligent machine None None None
Done
pattern recognition None None None
Done
computer vision None None None
Done
0               NaN
1      Santa Monica
2    Silicon Valley
3     Mountain View
4         Singapore
Name: location, dtype: object
